# Clean SNPs and pop structure output

This script will run a basic clean of SNPs on the merged HGDP-1000G reference files.
Specifically, we will remove SNPs with minor allele frequencies below 0.01, a Hardy-Weinberg equilibrium exact test p-value below 1e-50, missing call rates exceeding 0.1.
After that we will run an LD prune with two set of options.
One, "relaxed", using a window size of 50 SNPs, with a step size of 5 SNPs, and a variance inflation factor (VIF) of 2.
The other, "strict", using a window size of 500 SNPs, with a step size of 5 SNPs, and a variance inflation factor (VIF) of 1.3.
The relaxed will be used to run a PCA.
The strict will be used to obtain the allele frequencies for each population, and obtain an estimation of genetic distance.
It will be used to run an ADMIXTURE analysis as well.

In [1]:
#Importing modules
import subprocess, os, glob, shutil
import pandas as pd

In [2]:
#Setting paths
projpath   = os.path.realpath("..")
pathgeno   = os.path.join(projpath, "DataBases", "Genotypes")
pathclean  = os.path.join(projpath, "Results", "CleanGenos")
pathcounts = os.path.join(projpath, "Results", "Counts")
pathadmix  = os.path.join(projpath, "Results", "ADMIXTURE")
pathpca    = os.path.join(projpath, "Results", "PCA")
pathinfo   = os.path.join(projpath, "DataBases", "PopInfo")

In [9]:
#Cleaning genotyopes and output the log files

os.chdir(pathgeno)
#Removing MAF
subprocess.run(["plink", "--bfile", "hgdp1000ghg19", "--maf", "0.01", "--make-bed", "--out", "clean_temp1"])
#Removing missing call rates
subprocess.run(["plink", "--bfile", "clean_temp1", "--geno", "0.1", "--make-bed", "--out", "clean_temp2"])
#Removing H-W
subprocess.run(["plink", "--bfile", "clean_temp2", "--hwe", "1e-50", "--make-bed", "--out", "clean_temp3"])
#LD prune relaxed
subprocess.run(["plink", "--bfile", "clean_temp3", "--indep", "50", "5", "2", "--out", "relaxed"])
subprocess.run(["plink", "--bfile", "clean_temp3", "--extract", "relaxed.prune.in", "--make-bed", "--out", "CleanGenos_relaxed"])
#LD prune strict
subprocess.run(["plink", "--bfile", "clean_temp3", "--indep", "500", "5", "1.3", "--out", "strict"])
subprocess.run(["plink", "--bfile", "clean_temp3", "--extract", "strict.prune.in", "--make-bed", "--out", "CleanGenos_strict"])

#Read log files with the variants section
for file in glob.glob("*.log"):
    with open(file) as myfile:
        for num, line in enumerate(myfile, 1):
            if "variants" in line:
                print(line, end='')
        print("Finished file... \n")

641330 variants loaded from .bim file.
3589 variants removed due to missing genotype data (--geno).
637741 variants and 3444 people pass filters and QC.
Finished file... 

622661 variants loaded from .bim file.
--extract: 97941 variants remaining.
97941 variants and 3444 people pass filters and QC.
Finished file... 

622661 variants loaded from .bim file.
622661 variants and 3444 people pass filters and QC.
Pruned 40238 variants from chromosome 1, leaving 7848.
Pruned 44123 variants from chromosome 2, leaving 7688.
Pruned 36636 variants from chromosome 3, leaving 6528.
Pruned 32743 variants from chromosome 4, leaving 6013.
Pruned 32987 variants from chromosome 5, leaving 5992.
Pruned 35872 variants from chromosome 6, leaving 5958.
Pruned 29105 variants from chromosome 7, leaving 5242.
Pruned 30956 variants from chromosome 8, leaving 5095.
Pruned 25728 variants from chromosome 9, leaving 4598.
Pruned 28328 variants from chromosome 10, leaving 5148.
Pruned 26345 variants from chromosome 

In [10]:
#Creating populatio file, generating the allele counts

os.chdir(pathinfo)
#Creating an allele frequency file from CleanGenos_strict
#Create a within file for plink containing FID IID and cluster name
info1000g = pd.read_table("integrated_call_samples_v3.20130502.ALL.panel", header = None, skiprows = 1)
infohgdp  = pd.read_table("SampleInformation.txt", header = None, skiprows = 1, dtype = str)
#Fixing hgdp info file
infohgdp = infohgdp.drop(index = 1066)
for i in range(0,len(infohgdp)):
    n   = len(infohgdp.iloc[i,0])
    ze  = 5 - n
    add = "HGDP" + "0" * ze + infohgdp.iloc[i,0]
    infohgdp.iloc[i,0] = add

#Loading fam file
os.chdir(pathgeno)
fam = pd.read_table("hgdp1000ghg19.fam", sep = " ", header = None)
#Creating within file and writing
temp = pd.merge(fam, info1000g, how="left", left_on=1, right_on=0)
temp = pd.merge(temp, infohgdp, how="left", left_on=1, right_on=0)
pops = pd.concat([fam.iloc[:,0], fam.iloc[:,1], temp.iloc[:,9].fillna( temp.iloc[:,14]) ], axis = 1 )#+ test.iloc[:,30])
pops.to_csv("pops.txt", sep = " ", header=False, index=False)

#Running pop allele frequencies
subprocess.run(["plink", "--bfile", "CleanGenos_strict", "--within", "pops.txt", "--freq", "--out", "CleanGenos_strict"])

#PCA analysis
subprocess.run(["plink", "--bfile", "CleanGenos_relaxed", "--pca", "50", "--out", "CleanGenos_strict_PCA"])

CompletedProcess(args=['plink', '--bfile', 'CleanGenos_relaxed', '--pca', '50', '--out', 'CleanGenos_strict_PCA'], returncode=0)

In [11]:
#Changing from long to wide dataframe
os.chdir(os.path.join(projpath, "Code"))
subprocess.run(["bash", "LongToWide.sh"])

CompletedProcess(args=['bash', 'LongToWide.sh'], returncode=0)

In [12]:
os.chdir(pathgeno)
#Removing temp files
for file in glob.glob("*_temp*"):
    os.remove(file)
    
#Moving PCA files
for filename in glob.glob("*_PCA.*"):
    shutil.move(filename, pathpca)
    
#Moving count files
for filename in glob.glob("*.frq.*"):
    shutil.move(filename, pathcounts)

#Moving cleangenos files
for filename in glob.glob("CleanGenos_strict.*"):
    shutil.move(filename, pathclean)

#Moving cleangenos files
for filename in glob.glob("CleanGenos_relaxed.*"):
    shutil.move(filename, pathclean)

In [14]:
#ADMIXTURE analysis from k = 2-10 (takes a while)
refsamples = os.path.join(pathclean, "CleanGenos_strict.bed")
os.chdir(pathadmix)
for i in range(2,11):
    f = open("log_" + str(i) + ".txt", "w")
    subprocess.run(["./admixture", "--cv", refsamples, str(i)], stdout=f)

KeyboardInterrupt: 